In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymysql

In [2]:
# 영화 리스트 정적 웹 크롤링
url="https://movie.naver.com/movie/running/current.naver"
base="https://movie.naver.com"

res=requests.get(url).content
soup=bs(res, 'html.parser')
#print(soup)

movies=soup.select('div.lst_wrap > ul > li')
#print(movies[0].text)

# 영화 목록 저장하기 위한 빈 리스트 생성
movie_list=[]

for movie in movies:
    title=movie.select('dl > dt > a')[0].text
    rated=movie.select_one('dl > dt > span').text
    point=movie.select_one('dl > dd.star > dl > dd > div > a > span.num').text
    #print(title,rated,point)
    info=movie.select_one('dl.info_txt1 > dd').text
    #arr_info=info.split('|')
    #print(arr_info)
    arr_info=info.replace("\t","").replace("\n\n","").replace("\r\n","").split('|')
    if len(arr_info)<3:
        arr_info.insert(0,'장르정보 없음')
    genre=arr_info[0]
    running=arr_info[1].replace('분','')
    release_date=arr_info[2].replace('개봉','')
    #print(genre,running,release_date)
    
    director=movie.select_one('dl > dd:nth-child(3) > dl > dd:nth-child(4) > span').text.strip()
    #print(director)
    actor_list=movie.select('dl > dd:nth-child(3) > dl > dd:nth-child(6) > span > a')
    #print(actors)
    actorstr=''
    for actor in actor_list:
        actorstr+=actor.text+" "
    actors=actorstr.strip(',')
    
    # a 태그의 href 속성으로 영화 상세 페이지의 정보에 접근
    href=movie.select('dl > dt > a')[0]['href']
    url2=base+href
    res2=requests.get(url2).text
    soup2=bs(res2,'html.parser')
    story=soup2.select('div.section_group.section_group_frst > div:nth-child(1) > div > div.story_area > p')[0].text
    story=story.replace('\r','')
    #print(story)
    
    temp=soup2.select('#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph')
    if len(temp)>0:
        temp=temp[0].text.replace('\n','').replace('이상','').split('대')
        temp=temp[:-1]
        temp=' '.join(temp) # join()으로 하나의 문자열 생성
    else:
        temp='선호도 정보 없음'
    
    print(temp)
    
    movie_list.append((title,rated,point,genre,running,release_date,
                       director,actors,story,temp)) # tuple로 리스트 추가
    
#movie_list

선호도 정보 없음
선호도 정보 없음
3%10 30%20 44%30 17%40 6%50
20%10 0%20 40%30 40%40 0%50
14%10 29%20 14%30 14%40 29%50
0%10 40%20 33%30 13%40 13%50
선호도 정보 없음
선호도 정보 없음
6%10 17%20 28%30 28%40 22%50
0%10 25%20 25%30 25%40 25%50
선호도 정보 없음
선호도 정보 없음
1%10 14%20 20%30 41%40 25%50
0%10 0%20 50%30 50%40 0%50
0%10 53%20 26%30 16%40 5%50
0%10 17%20 26%30 30%40 26%50
0%10 38%20 29%30 14%40 19%50
0%10 0%20 60%30 40%40 0%50
0%10 27%20 47%30 20%40 7%50
3%10 33%20 39%30 19%40 5%50
0%10 0%20 17%30 58%40 25%50
0%10 30%20 35%30 27%40 8%50
선호도 정보 없음
0%10 14%20 43%30 29%40 14%50
0%10 18%20 36%30 36%40 9%50
0%10 26%20 43%30 21%40 11%50
0%10 25%20 0%30 25%40 50%50
0%10 22%20 44%30 22%40 11%50
4%10 17%20 48%30 22%40 9%50
0%10 5%20 37%30 53%40 5%50
0%10 0%20 100%30 0%40 0%50
3%10 37%20 39%30 17%40 3%50
1%10 40%20 36%30 17%40 6%50
2%10 32%20 33%30 23%40 10%50
0%10 50%20 12%30 38%40 0%50
0%10 0%20 62%30 38%40 0%50
2%10 32%20 35%30 21%40 9%50
5%10 41%20 17%30 29%40 8%50
0%10 0%20 75%30 25%40 0%50
2%10 39%20 31%30 20%40 8%50


In [9]:
df=pd.DataFrame(movie_list,
                columns=['title','rated','point','genre','running',
                         'release_date','director','actors','story','temp'])
df

,title,rated,point,genre,running,release_date,director,actors,story,temp
0,올리 마키의 가장 행복한 날,12세 관람가,0.00,"멜로/로맨스, 드라마",93,2022.06.02,유호 쿠오스마넨,에로 밀로노프 자코 라티 우나 아이롤라,평범한 제빵사에서 권투 영웅으로 떠오른 ‘올리’ 전 국민이 고대하는 세계 챔피언십을...,선호도 정보 없음
1,"괴물, 유령, 자유인",15세 관람가,0.00,"SF, 멜로/로맨스",77,2022.06.02,홍지영,엄태나 윤해신 권기하,성심과 은수는 동성연인이다. 사회에서는 결코 인정하지 않기에 결국 서로에 대한 믿음...,선호도 정보 없음
2,쥬라기 월드: 도미니언,12세 관람가,6.36,"액션, 모험",147,2022.06.01,콜린 트레보로우,크리스 프랫 브라이스 달라스 하워드,"공룡들의 터전이었던 이슬라 누블라 섬이 파괴된 후, 마침내 공룡들은 섬을 벗어나 세...",3%10 30%20 44%30 17%40 6%50
3,극장판 포켓몬스터DP: 기라티나와 하늘의 꽃다발 쉐이미,전체 관람가,7.45,애니메이션,96,2022.06.01,유야마 쿠니히코,이선호 김영선 마츠모토 리카 오오타니 이쿠에 우에다 유지,끝나지 않은 전설의 포켓몬들의 배틀로 위험에 빠진 반전 세계와 현실 세계를 구하기 ...,20%10 0%20 40%30 40%40 0%50
4,카시오페아,12세 관람가,9.37,드라마,102,2022.06.01,신연식,안성기 서현진 주예림,"이혼 후 변호사, 엄마로 완벽한 삶을 살아가려고 노력하는 수진은 하나뿐인 딸 지나의...",14%10 29%20 14%30 14%40 29%50
...,...,...,...,...,...,...,...,...,...,...
79,여름 이야기,12세 관람가,8.12,"코미디, 멜로/로맨스",114,1998.06.13,에릭 로메르,멜빌 푸포 아만다 랑글렛,가스파르(Gaspard: 멜빌 포포드 분)는 여름날 혼자 해변에 간다. 여자 친구 ...,10%10 26%20 58%30 3%40 3%50
80,로맨스 빠빠,15세 관람가,8.52,"가족, 멜로/로맨스",131,1960.01.28,신상옥,최은희 김진규 강신성일 김승호 주증녀,보험회사 사원인 그는 2남3녀의 아버지이다. 보험회사에 감원 선풍이 인다. 그는 건...,선호도 정보 없음
81,OK 목장의 결투,12세 관람가,8.68,"서부, 액션, 스릴러",117,1958.01.01,존 스터지스,버트 랭커스터 커크 더글러스,전직 치과의사였던 닥 할리데이(Dr. John 'Doc' Holliday: 커크 더...,선호도 정보 없음
82,젊은이의 양지,15세 관람가,8.81,"드라마, 느와르, 멜로/로맨스",122,1953.11.26,조지 스티븐스,몽고메리 클리프트 엘리자베스 테일러 쉘리 윈터스,조지 이스트맨(몽고메리 클리프트 분)은 가난하지만 잘 생기고 매력적이며 야망에 찬 ...,선호도 정보 없음


In [8]:
# mysql에 데이터 저장
conn=pymysql.connect(host='localhost',user='lyk',password='1234',db='pydb',charset='utf8')
cursor=conn.cursor()
sql="insert into movie(title,rated,point,genre,running,release_date,director,actors,story,temp) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
cursor.executemany(sql,movie_list)
conn.commit()
conn.close()

# error 발생시
# column 개수 맞추기
# mySQL에서 너무 긴 데이터일 경우 VARCHAR보다 TEXT를 사용해야 함
# mySQL 테이블 정보 변경시 jupyter lab 종료 후 시도해볼 것
# , \ | 와 같은 구분자는 삭제하거나 공백으로 바꿀 것
# 날짜 형식은 pandas에서 todate로 고쳐서 형식 지정

In [10]:
df=pd.DataFrame(movie_list,columns=['title','rated','point','genre','running',
                         'release_date','director','actors','story','temp'])
df.to_csv('data/movies_naver.csv', index=False)
df

,title,rated,point,genre,running,release_date,director,actors,story,temp
0,올리 마키의 가장 행복한 날,12세 관람가,0.00,"멜로/로맨스, 드라마",93,2022.06.02,유호 쿠오스마넨,에로 밀로노프 자코 라티 우나 아이롤라,평범한 제빵사에서 권투 영웅으로 떠오른 ‘올리’ 전 국민이 고대하는 세계 챔피언십을...,선호도 정보 없음
1,"괴물, 유령, 자유인",15세 관람가,0.00,"SF, 멜로/로맨스",77,2022.06.02,홍지영,엄태나 윤해신 권기하,성심과 은수는 동성연인이다. 사회에서는 결코 인정하지 않기에 결국 서로에 대한 믿음...,선호도 정보 없음
2,쥬라기 월드: 도미니언,12세 관람가,6.36,"액션, 모험",147,2022.06.01,콜린 트레보로우,크리스 프랫 브라이스 달라스 하워드,"공룡들의 터전이었던 이슬라 누블라 섬이 파괴된 후, 마침내 공룡들은 섬을 벗어나 세...",3%10 30%20 44%30 17%40 6%50
3,극장판 포켓몬스터DP: 기라티나와 하늘의 꽃다발 쉐이미,전체 관람가,7.45,애니메이션,96,2022.06.01,유야마 쿠니히코,이선호 김영선 마츠모토 리카 오오타니 이쿠에 우에다 유지,끝나지 않은 전설의 포켓몬들의 배틀로 위험에 빠진 반전 세계와 현실 세계를 구하기 ...,20%10 0%20 40%30 40%40 0%50
4,카시오페아,12세 관람가,9.37,드라마,102,2022.06.01,신연식,안성기 서현진 주예림,"이혼 후 변호사, 엄마로 완벽한 삶을 살아가려고 노력하는 수진은 하나뿐인 딸 지나의...",14%10 29%20 14%30 14%40 29%50
...,...,...,...,...,...,...,...,...,...,...
79,여름 이야기,12세 관람가,8.12,"코미디, 멜로/로맨스",114,1998.06.13,에릭 로메르,멜빌 푸포 아만다 랑글렛,가스파르(Gaspard: 멜빌 포포드 분)는 여름날 혼자 해변에 간다. 여자 친구 ...,10%10 26%20 58%30 3%40 3%50
80,로맨스 빠빠,15세 관람가,8.52,"가족, 멜로/로맨스",131,1960.01.28,신상옥,최은희 김진규 강신성일 김승호 주증녀,보험회사 사원인 그는 2남3녀의 아버지이다. 보험회사에 감원 선풍이 인다. 그는 건...,선호도 정보 없음
81,OK 목장의 결투,12세 관람가,8.68,"서부, 액션, 스릴러",117,1958.01.01,존 스터지스,버트 랭커스터 커크 더글러스,전직 치과의사였던 닥 할리데이(Dr. John 'Doc' Holliday: 커크 더...,선호도 정보 없음
82,젊은이의 양지,15세 관람가,8.81,"드라마, 느와르, 멜로/로맨스",122,1953.11.26,조지 스티븐스,몽고메리 클리프트 엘리자베스 테일러 쉘리 윈터스,조지 이스트맨(몽고메리 클리프트 분)은 가난하지만 잘 생기고 매력적이며 야망에 찬 ...,선호도 정보 없음
